In [ ]:
from data_loader import *
from utils import *

from concurrent.futures import ThreadPoolExecutor,as_completed
from itertools import product

In [ ]:
teams_data = pd.read_parquet(home_dir + "/data/Teams/team_league.parquet")

In [ ]:
unique_teams = teams_data['team_name'].unique()
print("Expected API requests: ",int(len(unique_teams) * 38 * .8))

In [ ]:
teams = unique_teams
season = [2022]
team_season_pair = list(product(teams,season))

In [ ]:
parallel = True
workers = 3
visited_list = list()

if parallel:
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(read_fixtures_for_season,t,s) : (t,s) for t,s in team_season_pair}
else:
    for team in teams:
        try:
            dat = read_fixtures_for_season(team,season)
            visited_list.append(team)
        except:
            print(f"{team} does not have matches in {season}")

In [ ]:
teams = list(set(teams) - set(visited_list))
teams

In [ ]:
teams